In [2]:
!pip install --upgrade twarc
!pip install twarc-csv --user
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import nltk, numpy, pandas, os, sys, random, time, twarc
import matplotlib.pyplot as plt
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/soldier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

BEARER: AAAAAAAAAAAAAAAAAAAAAFkmfgEAAAAADcgLX%2B3II8bI4oZ7dDWTGfqEsis%3DckGXQZEmb9y5fkhZR34eXvdatbiWSSg4vCxZla4pmfGXndZCD2   

API KEY: hPOI2LgsaW5vylwt0AmCXLHXA   

API SECRET: mcNe4tlYUKhmDOWmB4ixZN7qYSfTD71Z6KAU08rxtletApPxnr  

ACCESS TOKEN: 1430421232239796227-BctF5asyBRXL3xoLioeq7Q0iWeiQV0  

ACCESS SECRET: OK5KkazmfZSUWJoZ142trqFDac350T7H2bt7K1WxmTD8N


In [2]:
!twarc2 configure

Please enter your Bearer Token (leave blank to skip to API key configuration): ^C

Aborted!


In [1]:
# !twarc2 search "russia AND putin AND annex OR annexation OR annexes OR annexed AND ukraine OR eastern" ./ukrain_REST.jsonl --start-time 2022-09-30 --end-time 2022-10-2 --limit 1000000
!twarc2 search "(putin annex ukraine) OR (putin annex Kherson) OR (putin annex Luhansk) OR (putin annex Donetsk) OR (putin annex Zaporizhzhia)" ./ukrain_REST.jsonl --start-time 2022-09-30 --end-time 2022-10-3 --limit 1000000

Usage: twarc2 configure [OPTIONS]
Try 'twarc2 configure --help' for help.

Error: No such option: --start-time


In [16]:
#!twarc2 flatten /content/drive/MyDrive/ukraine_REST.jsonl /content/drive/MyDrive/ukraine_REST_flat.jsonl
#!twarc2 flatten /content/drive/MyDrive/ukraine_STREAM.jsonl /content/drive/MyDrive/ukraine_STREAM_flat.jsonl

100% 633M/633M [01:19<00:00, 8.34MB/s]
⚡ Unterminated string starting at: line 1 column 3807 (char 3806)


In [36]:
!pip3 install twarc-network
!twarc2 network /content/drive/MyDrive/ukraine_STREAM_flat.jsonl --format csv network.csv --min-component-size 30


Aborted!


In [4]:
stream_df = pd.read_csv('/content/drive/MyDrive/network_stream.csv', names=['Source', 'Target', 'Weight', 'Retweet', 'Reply', 'Quote', 'Mention'])
stream_df = stream_df.drop(['Mention'], axis=1)
stream_df['Weight'] = stream_df['Weight'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
stream_df['Retweet'] = stream_df['Retweet'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
stream_df['Reply'] = stream_df['Reply'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
stream_df['Quote'] = stream_df['Quote'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])

Mounted at /content/drive


In [8]:
net = nx.from_pandas_edgelist(stream_df, source="Source", target="Target", edge_attr='Weight', create_using=nx.DiGraph())
cugraph.draw(net, with_labels=True, node_size=60,font_size=8)
plt.figure(figsize=(200,200)) 
plt.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cugraph-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cugraph
  Running setup.py clean for cugraph
Failed to build cugraph
  error: subprocess-exited-with-error
  
  × Running setup.py install for cugraph did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for cugraph ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> cugraph

note: This is an issue with the package mentioned above, not pip.
hint: See above 

ModuleNotFoundError: ignored